<a href="https://colab.research.google.com/github/FilatovArtem/MTS_ML-Course/blob/main/HW_%D0%BB%D0%B8%D0%BD%D0%B5%D0%B8%CC%86%D0%BD%D1%8B%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание по регрессии

В разделе "4.3. Выбросы и модификация таргета" семинара мы с вами рассматривали датасет diabetes и пробовали популярные приемы для того чтобы лучше прогнозировать высокие значения (y > 270)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
x_full, y_full = load_diabetes(return_X_y=True)
n_samples, n_features = x_full.shape

Напомним какой получался MAPE на наблюдениях с большими значениями y:
*   MAPE без логарифмирования 0.24,
*   после логарифмирования 0.26,
*   после преобразования Бокса-Кокса 0.25

С MSE картина обстояла еще более драматично
*   без логарифмирования 5928.76
*   после логарифмирования 7614.72
*   после преобразования Бокса-Кокса 7028.17



**! Важно! На семинаре мы и обучались и измеряли качество на всей выборке. Здесь сначала поступим так же**

Что же делать в таком случае?       
Выходов не менее двух:


1. Использовать другую метрику чтобы оценивать результаты под другим углом -- это не изменит предсказания модели, но может оказаться, что модель и так уже приемлемо работает (MRAE, MDAPE, MASE, eB)
2. Использовать другую функцию потерь -- так, чтобы минимизировать ошибку в области больших значениях таргета. Здесь частный случай -- взвесить наблюдения пропорционально таргету


Давайте совместим оба пункта и в нашем SGD-классе будем оптимизировать сразу MAPE !

In [ ]:
rng = np.random.default_rng(seed=121)
class sgd_lecture_linear:
  def __init__(self, rng, MAX_ITER = 100_000):
    self.MAX_ITER = MAX_ITER
    self.w = None
    self.w0 = rng.normal()
    self.loss = None
    return

  def _f(self, x):
    assert len(x) == len(self.w)
    return np.dot(self.w, x) + self.w0

  def _loss_mse(self, x, y):
    return (y - self._f(x)) ** 2

  def _der_loss(self, x, y):
    if self.loss == 'MSE':
      return -(y - self._f(x)) * x

  def fit(self, X_train, y_train, loss = 'MSE'):
    self.loss = loss
    self.w = rng.normal(size=X_train.shape[1])
    step = 0.01
    for k in range(self.MAX_ITER):
      rand_index = rng.integers(0, X_train.shape[0] - 1)
      x = np.array(X_train)[rand_index]
      y = np.array(y_train)[rand_index]
      if k % 10000 == 0:
         step = step / 2
      self.partial_fit(x, y, step)

  def partial_fit(self, x, y, step = 0.01):
    loss = self.loss
    if not self.w.any():
      self.w = rng.normal(len(x))
    dl = self._der_loss(x, y)
    self.w -= step * dl
    if self.loss == 'MSE':
      self.w0 -= - step * (y - self._f(x))

  def predict_proba(self, x):
    x = np.array(x)
    preds = []
    pred_fuction = None

    if self.loss in ['MSE']:
      pred_fuction = self._f
    for x_curr in x:
      preds.append(pred_fuction(x_curr))
    return preds

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape

In [ ]:
sgd = sgd_lecture_linear(rng)
sgd.fit(x_full, y_full)
print(f'{mape(y_full, sgd.predict_proba(x_full)):.2f}')

0.50


Давайте посмотрим на MAPE

$$
MAPE\_loss = \sum_i{\frac{|y_i - a_i|}{|y_i|}}
$$

от MAE он отличается лишь весами, обратно пропорциональными таргету

$$
MAE\_loss = \sum_i{|y_i - a_i|}
$$

давайте обучим взвешенный MSE

In [ ]:
class sgd_lecture_linear:
  def __init__(self, rng, MAX_ITER = 50_000, sample_weights = None):
    self.MAX_ITER = MAX_ITER
    self.w = None
    self.w0 = rng.normal()
    self.loss = None
    self.epsilon = 1e-10
    return

  def _f(self, x):
    assert len(x) == len(self.w)
    return np.dot(self.w, x) + self.w0

  def _loss_mse(self, x, y):
    return (y - self._f(x)) ** 2

  def _der_loss(self, x, y, weight = 1):
      return -(y - self._f(x)) * x * weight

  def fit(self, X_train, y_train, loss = 'MSE', weights = None):
    self.loss = loss
    self.w = rng.normal(size=X_train.shape[1])
    step = 0.01
    if loss == 'MSE':
      for k in range(self.MAX_ITER):
        rand_index = rng.integers(0, X_train.shape[0] - 1)
        x = np.array(X_train)[rand_index]
        y = np.array(y_train)[rand_index]
        if k % 10000 == 0:
          step = step / 2
        self.partial_fit(x, y, step)
    elif loss == 'MSE_weighted':
      for k in range(self.MAX_ITER):
        rand_index = rng.integers(0, X_train.shape[0] - 1)
        x = np.array(X_train)[rand_index]
        y = np.array(y_train)[rand_index]
        weight = np.array(weights)[rand_index]
        if k % 10000 == 0:
          step = step / 2
        self.partial_fit(x, y, step, weight = weight)

  def partial_fit(self, x, y, step = 0.01, weight = 1):
    loss = self.loss
    if not self.w.any():
      self.w = rng.normal(len(x))
    dl = self._der_loss(x, y, weight = weight)
    self.w -= step * dl
    if self.loss == 'MSE':
      self.w0 -= - step * (y - self._f(x))
    if self.loss == 'MSE_weighted':
      self.w0 -= - step * (y - self._f(x)) * weight

  def predict_proba(self, x):
    x = np.array(x)
    preds = []
    pred_fuction = None

    if self.loss in ['MSE', 'MSE_weighted']:
      pred_fuction = self._f
    for x_curr in x:
      preds.append(pred_fuction(x_curr))
    return preds

In [ ]:
sgd_mape = sgd_lecture_linear(rng)
weights =  (y_full - min(y_full)) / (max(y_full) - min(y_full)) + 0.1
weights = 1 / weights
sgd_mape.fit(x_full, y_full, loss = 'MSE_weighted', weights = weights)
print(f'{mape(y_full, sgd_mape.predict_proba(x_full)):.2f}')

0.38


Ого! MAPE сильно упал! это здорово!

Давайте проверим что и в честном train-test split эффект сохранится

In [ ]:
df = pd.DataFrame(x_full)
df['target'] = y_full
test_size = 0.3
random_state = 43
x_train, x_test, y_train, y_test = train_test_split(df.drop(['target'], axis = 1), df['target'], test_size=test_size, random_state=random_state)

In [ ]:
sgd = sgd_lecture_linear(rng)
sgd.fit(x_train, y_train)
print(f'{mape(y_test, sgd.predict_proba(x_test)):.2f}', ' -- MAPE на тесте при обучении с обычным MSE')
sgd_mse_w = sgd_lecture_linear(rng)
weights =  (y_train - min(y_train)) / (max(y_train) - min(y_train)) + 0.03 # чтобы не делить на ноль
weights = 1 / weights
sgd_mse_w.fit(x_train, y_train, loss = 'MSE_weighted', weights = weights)
print(f'{mape(y_test, sgd_mse_w.predict_proba(x_test)):.2f}', ' -- MAPE на тесте при обучении на weighted MSE')

0.50  -- MAPE на тесте при обучении с обычным MSE
0.35  -- MAPE на тесте при обучении на weighted MSE


Давайте померяем как стали прогнозироваться объекты с большими значениями таргета

In [ ]:
df_test = pd.DataFrame(y_test, columns = ['target'])
df_test['pred_MSE'] = sgd.predict_proba(x_test)
df_test['pred_MSE_weighted'] = sgd_mse_w.predict_proba(x_test)
print(mape(df_test[df_test['target'] > 270]['target'], df_test[df_test['target'] > 270]['pred_MSE']), ' -- MAPE на тесте при обучении с обычным MSE на больших таргетах (>270)')
print(mape(df_test[df_test['target'] > 270]['target'], df_test[df_test['target'] > 270]['pred_MSE_weighted']), ' -- MAPE на тесте при обучении с weighted MSE на больших таргетах (>270)')

0.37244520573827306  -- MAPE на тесте при обучении с обычным MSE на больших таргетах (>270)
0.4487208396445101  -- MAPE на тесте при обучении с weighted MSE на больших таргетах (>270)


Стало даже хуже!   
Давайте попробуем поменять веса у таких объектов

In [ ]:
sgd = sgd_lecture_linear(rng)
sgd.fit(x_train, y_train)
print(f'{mape(y_test, sgd.predict_proba(x_test)):.2f}', ' -- MAPE на тесте при обучении с обычным MSE')
sgd_mse_w = sgd_lecture_linear(rng)
weights =  (y_train - min(y_train)) / (max(y_train) - min(y_train)) + 0.03 # чтобы не делить на ноль
weights = 1 / weights
for idx, y in zip(weights.index, y_train):
  if y > 270:
    weights[idx] = weights[idx] * 10

sgd_mse_w.fit(x_train, y_train, loss = 'MSE_weighted', weights = weights)
print(f'{mape(y_test, sgd_mse_w.predict_proba(x_test)):.2f}', ' -- MAPE на тесте при обучении на weighted MSE')

df_test = pd.DataFrame(y_test, columns = ['target'])
df_test['pred_MSE'] = sgd.predict_proba(x_test)
df_test['pred_MSE_weighted'] = sgd_mse_w.predict_proba(x_test)
mape_ordinary = mape(df_test[df_test['target'] > 270]['target'], df_test[df_test['target'] > 270]['pred_MSE'])
mape_weighted_mse = mape(df_test[df_test['target'] > 270]['target'], df_test[df_test['target'] > 270]['pred_MSE_weighted'])
print(f'{mape_ordinary:.2f}', ' -- MAPE на тесте (только на больших таргетах) при обучении с обычным MSE на больших таргетах (>270)')
print(f'{mape_weighted_mse:.2f}', ' -- MAPE на тесте (только на больших таргетах) при обучении с weighted MSE на больших таргетах (>270)')

0.50  -- MAPE на тесте при обучении с обычным MSE
0.38  -- MAPE на тесте при обучении на weighted MSE
0.37  -- MAPE на тесте (только на больших таргетах) при обучении с обычным MSE на больших таргетах (>270)
0.22  -- MAPE на тесте (только на больших таргетах) при обучении с weighted MSE на больших таргетах (>270)


**Кажется, ошибка на больших значениях существенно снизилась!**     
Давайте нарисуем это



In [ ]:
import plotly.express as px
import plotly.graph_objs as go

df = pd.DataFrame(dict(
    series=np.concatenate((["Таргет"] * len(y_test), ["Предсказание через оптимизацию MSE"] * len(y_test))),
    data=np.concatenate((y_test, df_test['pred_MSE']))
))


fig = px.histogram(df, x="data", color="series", barmode="overlay")
fig.update_layout(
                  title="Распределение таргетов и предиктов"
                  , xaxis_title="Таргеты"
                  , yaxis_title="Число"
                  , margin=dict(l=0, r=0, t=50, b=0)
                  , height=600
                  , width=900
                  , font_family="Arial"
                  , font_color="black"
                  , font_size = 20
                  , title_font_family="Times New Roman"
                  , title_font_size = 20
                  , title_font_color="black"
                  , separators=", .*"
)
fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

df = pd.DataFrame(dict(
    series=np.concatenate((["Таргет"] * len(y_test), ["Предсказание через взвешенный MSE"] * len(y_test))),
    data=np.concatenate((y_test, df_test['pred_MSE_weighted']))
))


fig = px.histogram(df, x="data", color="series", barmode="overlay")
fig.update_layout(
                  title="Распределение таргетов и предиктов"
                  , xaxis_title="Таргеты"
                  , yaxis_title="Число"
                  , margin=dict(l=0, r=0, t=50, b=0)
                  , height=600
                  , width=900
                  , font_family="Arial"
                  , font_color="black"
                  , font_size = 20
                  , title_font_family="Times New Roman"
                  , title_font_size = 20
                  , title_font_color="black"
                  , separators=", .*"
)
fig.show()

## Само задание: попробуйте обучиться сразу на MAPE и измерьте полученный MAPE на всей тестовой выборке

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape

x_full, y_full = load_diabetes(return_X_y=True)
df = pd.DataFrame(x_full)
df['target'] = y_full

test_size = 0.3
random_state = 43
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(['target'], axis=1), df['target'],
    test_size=test_size, random_state=random_state
)

scale_factor = np.mean(y_train)
y_train_scaled = y_train / scale_factor

rng = np.random.default_rng(seed=121)


class sgd_lecture_linear:
    def __init__(self, rng, MAX_ITER=100_000, sample_weights=None,
                 early_stopping=False, validation_fraction=0.1,
                 n_iter_no_change=5000, tol=1e-4):
        self.MAX_ITER = MAX_ITER
        self.w = None
        self.w0 = rng.normal()
        self.loss = None
        self.epsilon = 1e-10
        self.rng = rng
        self.early_stopping = early_stopping
        self.validation_fraction = validation_fraction
        self.n_iter_no_change = n_iter_no_change
        self.tol = tol
        return

    def _f(self, x):
        assert len(x) == len(self.w)
        return np.dot(self.w, x) + self.w0

    def _compute_loss(self, X, y, weights=None):
        predictions = self.predict_proba(X)
        if weights is None:
            weights = np.ones(len(y))
        if self.loss in ['MSE', 'MSE_weighted']:
            return np.mean(weights * (y - predictions) ** 2)
        elif self.loss == 'MAPE':
            abs_errors = np.abs((y - predictions) / (np.abs(y) + self.epsilon))
            return np.mean(weights * abs_errors) * 100

    def _der_loss(self, x, y, weight=1):
        if self.loss in ['MSE', 'MSE_weighted']:
            return -(y - self._f(x)) * x * weight
        elif self.loss == 'MAPE':
            error = y - self._f(x)
            abs_y = np.abs(y) + self.epsilon
            if error >= 0:
                return - (1 / abs_y) * x * weight
            else:
                return (1 / abs_y) * x * weight

    def fit(self, X_train, y_train, loss='MSE', weights=None):
        self.loss = loss
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        if self.early_stopping:
            n_samples = X_train.shape[0]
            n_val = int(n_samples * self.validation_fraction)
            indices = self.rng.permutation(n_samples)
            val_indices = indices[:n_val]
            train_indices = indices[n_val:]

            X_val = X_train[val_indices]
            y_val = y_train[val_indices]
            X_train = X_train[train_indices]
            y_train = y_train[train_indices]

            if weights is not None:
                weights = np.array(weights)
                weights_val = weights[val_indices]
                weights = weights[train_indices]
            else:
                weights_val = None

            best_val_loss = float('inf')
            best_weights = None
            best_w0 = None
            iterations_no_change = 0

        self.w = self.rng.normal(size=X_train.shape[1])
        step = 0.01

        if loss == 'MSE':
            for k in range(self.MAX_ITER):
                rand_index = self.rng.integers(0, X_train.shape[0] - 1)
                x = X_train[rand_index]
                y = y_train[rand_index]
                if k % 20000 == 0 and k != 0:
                    step = step / 2
                self.partial_fit(x, y, step)

                if self.early_stopping and k % 1000 == 0:
                    val_loss = self._compute_loss(X_val, y_val, weights_val)
                    if val_loss < best_val_loss - self.tol:
                        best_val_loss = val_loss
                        best_weights = self.w.copy()
                        best_w0 = self.w0
                        iterations_no_change = 0
                    else:
                        iterations_no_change += 1000
                    if iterations_no_change >= self.n_iter_no_change:
                        self.w = best_weights
                        self.w0 = best_w0
                        break

        elif loss in ['MSE_weighted', 'MAPE']:
            for k in range(self.MAX_ITER):
                rand_index = self.rng.integers(0, X_train.shape[0] - 1)
                x = X_train[rand_index]
                y = y_train[rand_index]
                weight = np.array(weights)[rand_index] if weights is not None else 1
                if k % 20000 == 0 and k != 0:
                    step = step / 2
                self.partial_fit(x, y, step, weight=weight)

                if self.early_stopping and k % 1000 == 0:
                    val_loss = self._compute_loss(X_val, y_val, weights_val)
                    if val_loss < best_val_loss - self.tol:
                        best_val_loss = val_loss
                        best_weights = self.w.copy()
                        best_w0 = self.w0
                        iterations_no_change = 0
                    else:
                        iterations_no_change += 1000
                    if iterations_no_change >= self.n_iter_no_change:
                        self.w = best_weights
                        self.w0 = best_w0
                        break

    def partial_fit(self, x, y, step=0.01, weight=1):
        if self.w is None:
            self.w = self.rng.normal(size=len(x))
        dl = self._der_loss(x, y, weight=weight)
        self.w -= step * dl
        if self.loss == 'MSE':
            self.w0 -= - step * (y - self._f(x))
        elif self.loss == 'MSE_weighted':
            self.w0 -= - step * (y - self._f(x)) * weight
        elif self.loss == 'MAPE':
            error = y - self._f(x)
            abs_y = np.abs(y) + self.epsilon
            if error >= 0:
                grad_w0 = - (1 / abs_y) * weight
            else:
                grad_w0 = (1 / abs_y) * weight
            self.w0 -= step * grad_w0

    def predict_proba(self, X):
        X = np.array(X)
        if X.ndim == 1:
            return self._f(X)
        return X @ self.w + self.w0


sgd_mape = sgd_lecture_linear(rng, MAX_ITER=100_000, early_stopping=True)
weights = np.ones(len(y_train_scaled))
sgd_mape.fit(x_train, y_train_scaled, loss='MAPE', weights=weights)
pred_scaled = sgd_mape.predict_proba(x_test)
pred = np.array(pred_scaled) * scale_factor
mape_value = mape(y_test, pred)
print(f'{mape_value:.2f}', ' -- MAPE на тесте при обучении на MAPE')

0.34  -- MAPE на тесте при обучении на MAPE


Вопрос: в какой интервал попадает MAPE на тесте при обучении на MAPE?



1.   0.34 -- 0.37 <-----
2.   0.37 -- 0.40
3.   0.41 -- 0.43
4.   0.44 -- 0.47


